# Experiment 1: Cross-Model Semantic Leakage Benchmark

**Research Question:** Which LLM produces the most logically consistent biomedical Chain-of-Thought reasoning?

**What we measure:**
- Contradiction rate between adjacent reasoning steps (per model)
- How contradiction rate grows with reasoning depth (step index)
- UMLS concept validity rate and its correlation with logical consistency
- Guard signal distribution across models

**Pipeline:** Question → LLM CoT → UMLS Concept Extraction → Hybrid NLI Entailment → Analysis

Results are cached to `results/exp1_*.json` so re-running the notebook skips expensive API calls.

In [ ]:
# ============================================================
# SETUP: Clone repo, install deps, set API keys
# Run this cell first — works in Colab and local Jupyter
# ============================================================
import os, sys
from pathlib import Path

# ── 1. Clone or update the repository ───────────────────────
REPO_URL  = 'https://github.com/varchanaiyer/biomedical-semantic-leakage-detection'
REPO_DIR  = 'biomedical-semantic-leakage-detection'

if not Path(REPO_DIR).exists():
    os.system(f'git clone {REPO_URL}')
else:
    os.system(f'git -C {REPO_DIR} pull --quiet')

# ── 2. Add project root to path ─────────────────────────────
_cwd = Path(os.getcwd())
if (_cwd / REPO_DIR / 'utils').exists():
    PROJECT_ROOT = str(_cwd / REPO_DIR)
elif (_cwd / 'utils').exists():
    PROJECT_ROOT = str(_cwd)
elif (_cwd.parent / 'utils').exists():
    PROJECT_ROOT = str(_cwd.parent)
else:
    PROJECT_ROOT = str(_cwd / REPO_DIR)  # fallback

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
os.chdir(PROJECT_ROOT)
print(f'PROJECT_ROOT: {PROJECT_ROOT}')

# ── 3. Install dependencies ──────────────────────────────────
os.system('pip install openai numpy pandas scipy scikit-learn matplotlib seaborn requests jupyter ipywidgets python-dotenv --quiet')

# ── 4. Set API keys directly ─────────────────────────────────
os.environ["UMLS_API_KEY"]       = "YOUR_UMLS_API_KEY_HERE"
os.environ["OPENROUTER_API_KEY"] = "YOUR_OPENROUTER_API_KEY_HERE"

print('Setup complete. API keys configured:', {
    k: ('set' if os.environ.get(k) else 'NOT SET')
    for k in ['OPENROUTER_API_KEY', 'UMLS_API_KEY']
})

In [ ]:
# ── OpenRouter API Key ────────────────────────────────────────────────────────
import os, importlib.util
from IPython.display import display, clear_output, HTML

_HAS_WIDGETS = importlib.util.find_spec("ipywidgets") is not None

if _HAS_WIDGETS:
    import ipywidgets as widgets

    _key_box = widgets.Password(
        placeholder="sk-or-v1-…  (get yours free at openrouter.ai)",
        layout=widgets.Layout(width="520px"),
    )
    _btn = widgets.Button(
        description="Set Key", button_style="primary",
        icon="check", layout=widgets.Layout(width="110px"),
    )
    _out = widgets.Output()

    def _apply(_b):
        with _out:
            clear_output()
            key = _key_box.value.strip()
            if key:
                os.environ["OPENROUTER_API_KEY"] = key
                print(f"  ✓ OpenRouter key set ({len(key)} chars)")
            else:
                print("  ✗ Paste your OpenRouter key above, then click Set Key.")

    _btn.on_click(_apply)
    display(HTML("<b>🔑 OpenRouter API Key</b>"))
    display(widgets.HBox([_key_box, _btn]))
    display(_out)
    display(HTML(
        "<small>Get a free key at "
        "<a href=\"https://openrouter.ai\" target=\"_blank\">openrouter.ai</a>"
        " — the notebooks will automatically run across all configured models.</small>"
    ))
else:
    os.environ.setdefault("OPENROUTER_API_KEY", "")
    print("ipywidgets not found — set key with:")
    print("  os.environ[\"OPENROUTER_API_KEY\"] = \"sk-or-v1-...\"")


In [ ]:
import sys, os, json, time, pickle
from pathlib import Path

# Add project root to path
# Project root (setup cell already set CWD and sys.path; this is a fallback for local use)
_cwd = Path(os.getcwd())
PROJECT_ROOT = _cwd if (_cwd / 'utils').exists() else _cwd.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
RESULTS_DIR = PROJECT_ROOT / 'experiments' / 'results'
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Use heuristic NLI — fast, no HuggingFace model download needed.
# Set to False only for final camera-ready runs (requires ~800 MB download).
USE_HEURISTIC_NLI = True
if USE_HEURISTIC_NLI:
    os.environ['FORCE_HEURISTIC_NLI'] = '1'

print(f'Project root: {PROJECT_ROOT}')
print(f'Results dir:  {RESULTS_DIR.resolve()}')
print(f'Heuristic NLI: {USE_HEURISTIC_NLI}')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from utils.cot_generator import generate as generate_cot
from utils.concept_extractor import extract_concepts
from utils.hybrid_checker import build_entailment_records
from utils.guards import derive_guards, GuardConfig
from utils.umls_api_linker import is_configured as umls_configured

print('All modules imported successfully.')
print(f'UMLS configured: {umls_configured()}')

In [ ]:
# ── 15 Diverse Biomedical Questions ──────────────────────────────────────────
# Representative subset covering drugs, diseases, mechanisms, diagnostics, treatments.
# 15 questions × 2 models = 30 API calls (fast enough for iterative runs).
# Scale to 40 questions × 4 models for camera-ready by uncommenting extras below.

QUESTIONS = [
    # Drug mechanisms
    "Does aspirin reduce the risk of myocardial infarction in patients with cardiovascular disease?",
    "What is the mechanism by which metformin lowers blood glucose in type 2 diabetes?",
    "How do statins reduce LDL cholesterol levels and cardiovascular risk?",
    # Disease processes
    "What is the pathophysiology of atherosclerosis leading to coronary artery disease?",
    "How does type 2 diabetes lead to peripheral neuropathy?",
    "What is the mechanism of hypertension-induced end-organ damage?",
    # Diagnostics
    "What are the diagnostic criteria for sepsis and how should it be managed?",
    "What biomarkers are used to diagnose acute myocardial infarction?",
    "How is systemic lupus erythematosus diagnosed using laboratory tests?",
    # Treatments
    "What is the first-line treatment for community-acquired pneumonia in outpatients?",
    "How is atrial fibrillation managed to prevent thromboembolic complications?",
    "What is the role of immunotherapy in treating non-small cell lung cancer?",
    # Drug interactions / multi-step
    "What are the risks of combining NSAIDs with anticoagulants?",
    "What is the connection between obesity, insulin resistance, and type 2 diabetes?",
    "How does the renin-angiotensin-aldosterone system contribute to hypertension?",
]

print(f'Total questions: {len(QUESTIONS)}')

In [ ]:
# ── Pipeline Runner ───────────────────────────────────────────────────────────

GUARD_CFG = GuardConfig()

def run_full_pipeline(question: str, prefer: str = 'openrouter',
                      model: str = None,
                      scispacy_when: str = 'never', top_k: int = 3) -> dict:
    """Run the full pipeline on a single question."""
    import time
    t0 = time.time()

    # Step 1: CoT generation
    cot      = generate_cot(question, prefer=prefer, model=model)
    steps    = cot.get('steps', [])
    provider = cot.get('provider', 'unknown')
    model_id = cot.get('model', model or 'unknown')

    # Step 2: Concept extraction (UMLS)
    concepts = extract_concepts(steps, scispacy_when=scispacy_when, top_k=top_k)

    # Step 3: Hybrid NLI entailment
    pairs = build_entailment_records(steps, concepts)

    # Step 4: Guard signals + UMLS relation metadata
    guarded_pairs = []
    for p in pairs:
        i, j = p['step_pair']
        si = steps[i] if i < len(steps) else ''
        sj = steps[j] if j < len(steps) else ''

        meta = p.get('meta', {})
        guards = derive_guards(
            premise       = si,
            hypothesis    = sj,
            probs         = p['probs'],
            reverse_probs = None,
            relation_violation = bool(meta.get('relation_violation', False)),
            ontology_override_signal = bool(meta.get('ontology_support', False)),
            config        = GUARD_CFG,
        )
        guarded_pairs.append({**p, 'guards': guards})

    return {
        'question':   question,
        'provider':   provider,
        'model':      model_id,
        'steps':      steps,
        'concepts':   [[{k: v for k, v in c.items() if k != 'scores'} | {'confidence': (c.get('scores') or {}).get('confidence', 0.0)}
                        for c in step_cands] for step_cands in concepts],
        'pairs':      guarded_pairs,
        'duration_s': round(time.time() - t0, 2),
        'errors':     cot.get('errors', []),
    }

print('Pipeline runner defined.')

In [ ]:
# ── Run Pipeline Across Models via OpenRouter ─────────────────────────────────
# Using 2 models for fast iteration. Add more for camera-ready.

OPENROUTER_MODELS = {
    'claude-haiku':   'anthropic/claude-haiku-4-5',
    'gpt-4o-mini':    'openai/gpt-4o-mini',
    # Uncomment for full benchmark:
    # 'gemini-flash':   'google/gemini-flash-1.5',
    # 'llama-3-70b':    'meta-llama/llama-3.3-70b-instruct',
}

SLEEP_BETWEEN_CALLS = 0.2
N_QUESTIONS = len(QUESTIONS)

# FORCE_RERUN = True to regenerate after pipeline fixes; set False once results look good.
FORCE_RERUN = True

def _cache_has_umls(results):
    """Check if cached results contain real UMLS concept data."""
    for r in results:
        for step_concepts in r.get('concepts', []):
            for c in step_concepts:
                if c.get('cui') or c.get('valid'):
                    return True
    return False

all_results = {}

for label, model_slug in OPENROUTER_MODELS.items():
    cache_path = RESULTS_DIR / f'exp1_{label}_results.json'

    if cache_path.exists() and not FORCE_RERUN:
        with open(cache_path) as f:
            cached = json.load(f)

        if umls_configured() and not _cache_has_umls(cached):
            print(f'[{label}] Cache has NO UMLS data — deleting stale cache')
            cache_path.unlink()
        else:
            print(f'[{label}] Loading cached results from {cache_path}')
            all_results[label] = cached
            print(f'  Loaded {len(cached)} results')
            continue
    elif cache_path.exists() and FORCE_RERUN:
        print(f'[{label}] FORCE_RERUN=True — deleting stale cache')
        cache_path.unlink()

    print(f'\n[{label}] ({model_slug}) — running {N_QUESTIONS} questions...')
    results = []

    for i, q in enumerate(QUESTIONS[:N_QUESTIONS]):
        try:
            r = run_full_pipeline(q, prefer='openrouter', model=model_slug)
            results.append(r)
            label_counts = {lbl: sum(1 for p in r['pairs'] if p['final_label'] == lbl)
                            for lbl in ['entailment', 'neutral', 'contradiction']}
            print(f'  [{i+1}/{N_QUESTIONS}] {q[:50]}...'
                  f'  steps={len(r["steps"])} {label_counts}')
        except Exception as e:
            print(f'  [{i+1}] ERROR: {e}')
            results.append({'question': q, 'provider': 'openrouter', 'model': model_slug,
                            'steps': [], 'concepts': [], 'pairs': [],
                            'duration_s': 0, 'errors': [str(e)]})
        time.sleep(SLEEP_BETWEEN_CALLS)

    with open(cache_path, 'w') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    print(f'  Saved to {cache_path}')
    all_results[label] = results

print(f'\nAll models done. ({len(all_results)} models × {N_QUESTIONS} questions)')
print(f'UMLS configured: {umls_configured()}')

In [ ]:
# ── Build Analysis DataFrame ──────────────────────────────────────────────────

import pandas as pd
import numpy as np

rows = []
pair_rows = []  # one row per step-pair (for depth analysis)

for prefer, results in all_results.items():
    for r in results:
        pairs   = r.get('pairs', [])
        steps   = r.get('steps', [])
        concepts = r.get('concepts', [])
        
        if not steps:
            continue
        
        n_pairs  = len(pairs)
        n_contra = sum(1 for p in pairs if p.get('final_label') == 'contradiction')
        n_entail = sum(1 for p in pairs if p.get('final_label') == 'entailment')
        n_neutral = sum(1 for p in pairs if p.get('final_label') == 'neutral')
        
        # Concept validity
        all_cands = [c for step_cands in concepts for c in step_cands]
        total_cands  = len(all_cands)
        valid_cands  = sum(1 for c in all_cands if c.get('valid'))
        steps_with_valid = sum(1 for sc in concepts if any(c.get('valid') for c in sc))
        
        # Guard signal counts
        all_guards = [g for p in pairs for g in p.get('guards', [])]
        
        # Avg NLI probs
        avg_p_contra = np.mean([p.get('probs', {}).get('contradiction', 0) for p in pairs]) if pairs else np.nan
        avg_p_entail = np.mean([p.get('probs', {}).get('entailment', 0) for p in pairs]) if pairs else np.nan
        
        rows.append({
            'model_prefer': prefer,
            'model_actual': r.get('model', prefer),
            'question': r['question'][:70],
            'n_steps': len(steps),
            'n_pairs': n_pairs,
            'n_contradiction': n_contra,
            'n_entailment': n_entail,
            'n_neutral': n_neutral,
            'contradiction_rate': n_contra / n_pairs if n_pairs else np.nan,
            'entailment_rate': n_entail / n_pairs if n_pairs else np.nan,
            'concepts_total': total_cands,
            'concepts_valid': valid_cands,
            'concept_valid_rate': valid_cands / total_cands if total_cands else np.nan,
            'steps_with_valid_concept': steps_with_valid,
            'step_coverage_rate': steps_with_valid / len(steps) if steps else np.nan,
            'n_guards_total': len(all_guards),
            'n_caution_band': all_guards.count('caution_band'),
            'n_lexical_dup': all_guards.count('lexical_duplicate'),
            'n_direction_conflict': all_guards.count('direction_conflict'),
            'avg_prob_contradiction': avg_p_contra,
            'avg_prob_entailment': avg_p_entail,
            'duration_s': r.get('duration_s', 0),
            'has_error': bool(r.get('errors')),
        })
        
        # Per-pair rows for depth analysis
        for p in pairs:
            depth = p.get('step_pair', [0, 1])[0]  # i index = depth
            pair_rows.append({
                'model_prefer': prefer,
                'question': r['question'][:50],
                'depth': depth,
                'label': p.get('final_label', 'unknown'),
                'prob_contradiction': p.get('probs', {}).get('contradiction', 0),
                'prob_entailment': p.get('probs', {}).get('entailment', 0),
                'prob_neutral': p.get('probs', {}).get('neutral', 0),
                'guards': '|'.join(p.get('guards', [])),
                'umls_jaccard': p.get('meta', {}).get('umls_overlap_jaccard', 0),
            })

df = pd.DataFrame(rows)
df_pairs = pd.DataFrame(pair_rows)

print(f'Summary DataFrame: {len(df)} questions x {len(df.columns)} columns')
print(f'Pairs DataFrame:   {len(df_pairs)} pairs x {len(df_pairs.columns)} columns')

# Save
df.to_csv(RESULTS_DIR / 'exp1_summary.csv', index=False)
df_pairs.to_csv(RESULTS_DIR / 'exp1_pairs.csv', index=False)
print('Saved CSVs.')

In [ ]:
# ── Table 1: Per-Model Summary Statistics ────────────────────────────────────

summary = df.groupby('model_prefer').agg(
    n_questions       = ('question', 'count'),
    avg_steps         = ('n_steps', 'mean'),
    avg_pairs         = ('n_pairs', 'mean'),
    contradiction_rate = ('contradiction_rate', 'mean'),
    entailment_rate   = ('entailment_rate', 'mean'),
    concept_valid_rate = ('concept_valid_rate', 'mean'),
    step_coverage_rate = ('step_coverage_rate', 'mean'),
    avg_prob_contra   = ('avg_prob_contradiction', 'mean'),
    avg_prob_entail   = ('avg_prob_entailment', 'mean'),
    caution_band_rate = ('n_caution_band', 'mean'),
).round(4)

print('=== Table 1: Per-Model Summary ===')
print(summary.T.to_string())
summary.to_csv(RESULTS_DIR / 'exp1_model_summary.csv')

In [ ]:
# ── Figure 1: Contradiction Rate per Model ────────────────────────────────────

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

models = df['model_prefer'].unique()
colors = ['#4C72B0', '#DD8452', '#55A868', '#C44E52']
color_map = dict(zip(models, colors))

# (a) Contradiction rate distribution per model
ax = axes[0]
for model in models:
    sub = df[df['model_prefer'] == model]['contradiction_rate'].dropna()
    ax.boxplot(sub, positions=[list(models).index(model)], widths=0.5,
               patch_artist=True,
               boxprops=dict(facecolor=color_map[model], alpha=0.7))
ax.set_xticks(range(len(models)))
ax.set_xticklabels(models)
ax.set_ylabel('Contradiction Rate per Question')
ax.set_title('(a) Contradiction Rate Distribution')
ax.axhline(0, color='grey', lw=0.5, linestyle='--')

# (b) Label breakdown stacked bar
ax = axes[1]
bar_data = df.groupby('model_prefer')[['n_contradiction', 'n_neutral', 'n_entailment']].mean()
bar_data.plot(kind='bar', stacked=True, ax=ax,
              color=['#C44E52', '#8172B2', '#4C72B0'], alpha=0.85)
ax.set_xlabel('Model')
ax.set_ylabel('Avg. # Pairs per Question')
ax.set_title('(b) Label Breakdown (avg per question)')
ax.legend(['Contradiction', 'Neutral', 'Entailment'], loc='upper right')
ax.tick_params(axis='x', rotation=0)

# (c) Concept valid rate vs contradiction rate scatter
ax = axes[2]
clean = df.dropna(subset=['concept_valid_rate', 'contradiction_rate'])
if clean.empty or clean['concept_valid_rate'].isna().all():
    ax.text(0.5, 0.5,
            'UMLS not configured\nSet UMLS_API_KEY to enable\nconcept validity scoring',
            ha='center', va='center', transform=ax.transAxes,
            fontsize=10, color='grey', style='italic',
            bbox=dict(boxstyle='round,pad=0.5', facecolor='#f0f0f0', alpha=0.8))
    ax.set_title('(c) Concept Validity vs. Contradiction Rate\n(UMLS required)')
else:
    for model in models:
        sub = clean[clean['model_prefer'] == model]
        ax.scatter(sub['concept_valid_rate'], sub['contradiction_rate'],
                   label=model, alpha=0.6, s=40, color=color_map[model])
    ax.set_xlabel('Concept Validity Rate (UMLS)')
    ax.set_ylabel('Contradiction Rate')
    ax.set_title('(c) Concept Validity vs. Contradiction Rate')
    ax.legend()

plt.suptitle('Experiment 1: Cross-Model Semantic Leakage Benchmark', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'exp1_fig1_model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
print('Figure 1 saved.')

In [ ]:
# ── Figure 2: Contradiction Rate by Reasoning Depth ───────────────────────────

from scipy import stats as scipy_stats

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# (a) Contradiction rate by depth per model
ax = axes[0]
max_depth = df_pairs['depth'].max()

for model in models:
    sub = df_pairs[df_pairs['model_prefer'] == model]
    depth_rates = (sub.groupby('depth')['label']
                   .apply(lambda x: (x == 'contradiction').mean())
                   .reset_index(name='contra_rate'))
    ax.plot(depth_rates['depth'], depth_rates['contra_rate'],
            marker='o', label=model, color=color_map.get(model), linewidth=2)

ax.set_xlabel('Step Pair Depth (i → i+1 position)')
ax.set_ylabel('Contradiction Rate')
ax.set_title('(a) Contradiction Rate by Reasoning Depth')
ax.legend()
ax.grid(True, alpha=0.3)

# (b) Avg P(contradiction) by depth — all models combined
ax = axes[1]
for model in models:
    sub = df_pairs[df_pairs['model_prefer'] == model]
    depth_probs = sub.groupby('depth')['prob_contradiction'].mean().reset_index()
    ax.plot(depth_probs['depth'], depth_probs['prob_contradiction'],
            marker='s', label=model, color=color_map.get(model), linewidth=2, linestyle='--')

ax.set_xlabel('Step Pair Depth')
ax.set_ylabel('Avg P(contradiction)')
ax.set_title('(b) Average P(contradiction) by Depth')
ax.legend()
ax.grid(True, alpha=0.3)

plt.suptitle('Semantic Leakage Grows with Reasoning Depth', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'exp1_fig2_leakage_by_depth.png', dpi=150, bbox_inches='tight')
plt.show()
print('Figure 2 saved.')

In [ ]:
# ── Figure 3: Guard Signal Distribution ──────────────────────────────────────
import numpy as np

GUARD_DISPLAY = {
    'n_caution_band':      ('caution_band',      '#4C72B0'),
    'n_lexical_dup':       ('lexical_duplicate',  '#DD8452'),
    'n_direction_conflict': ('direction_conflict', '#55A868'),
}

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# ── (a) Guard frequency per model ─────────────────────────────────────────────
ax = axes[0]
x      = np.arange(len(models))
width  = 0.25
offset = -(len(GUARD_DISPLAY) - 1) * width / 2

for k, (col, (label, color)) in enumerate(GUARD_DISPLAY.items()):
    means = [df[df['model_prefer'] == m][col].mean() for m in models]
    bars  = ax.bar(x + offset + k * width, means, width, label=label,
                   color=color, alpha=0.85)
    # annotate "0" on genuinely zero bars so they're not invisible
    for bar, val in zip(bars, means):
        if val < 0.05:
            ax.text(bar.get_x() + bar.get_width() / 2, 0.05,
                    '0', ha='center', va='bottom', fontsize=7, color='grey')

ax.set_xticks(x)
ax.set_xticklabels(models)
ax.set_xlabel('Model')
ax.set_ylabel('Avg. # Guards per Question')
ax.set_title('(a) Guard Signal Frequency per Model')
ax.legend()
ax.set_ylim(bottom=0)

# Note if direction_conflict is all zero (requires bidirectional NLI + data)
if df['n_direction_conflict'].max() == 0:
    ax.annotate('direction_conflict requires\nbidirectional NLI scoring',
                xy=(0.5, 0.95), xycoords='axes fraction',
                ha='center', va='top', fontsize=7,
                color='grey', style='italic')

# ── (b) Guard rate: contradiction pairs vs all other pairs ───────────────────
ax = axes[1]

is_contra    = df_pairs['label'] == 'contradiction'
n_contra     = is_contra.sum()
n_other      = (~is_contra).sum()

guard_names  = ['caution_band', 'direction_conflict']
guard_colors = ['#4C72B0', '#55A868']
bar_width    = 0.35

for gi, (guard, color) in enumerate(zip(guard_names, guard_colors)):
    has_guard = df_pairs['guards'].str.contains(guard, na=False)

    rate_contra = has_guard[is_contra].mean()  if n_contra > 0 else 0.0
    rate_other  = has_guard[~is_contra].mean() if n_other  > 0 else 0.0

    xs = np.array([gi * 2, gi * 2 + 1], dtype=float)
    ax.bar(xs[0], rate_contra, bar_width, color=color, alpha=0.85,
           label=f'{guard} (n={int(has_guard[is_contra].sum())})')
    ax.bar(xs[1], rate_other,  bar_width, color=color, alpha=0.45)

    for x_pos, rate in [(xs[0], rate_contra), (xs[1], rate_other)]:
        if rate < 0.01:
            ax.text(x_pos, 0.005, '0', ha='center', va='bottom',
                    fontsize=8, color='grey')

ax.set_xticks([0, 1, 2, 3])
ax.set_xticklabels(['caution_band\n(contra)', 'caution_band\n(other)',
                     'dir_conflict\n(contra)', 'dir_conflict\n(other)'])
ax.set_ylabel('Fraction of Pairs with Guard')
ax.set_title(f'(b) Guard Rate: Contradiction vs. Other Pairs\n'
             f'(contradiction pairs n={n_contra}, other n={n_other})')
ax.legend(fontsize=8)
ax.set_ylim(0, 1)

if n_contra == 0:
    ax.text(0.5, 0.5,
            'No contradiction pairs\nin this dataset',
            ha='center', va='center', transform=ax.transAxes,
            fontsize=10, color='grey', style='italic')

plt.suptitle('Guard Signal Analysis', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'exp1_fig3_guard_signals.png', dpi=150, bbox_inches='tight')
plt.show()

# Print diagnostic
print("Guard firing rates:")
for guard in ['caution_band', 'lexical_duplicate', 'direction_conflict']:
    has = df_pairs['guards'].str.contains(guard, na=False)
    print(f"  {guard:22s}: {has.mean():.3f} overall  "
          f"(contra={has[is_contra].mean():.3f}  other={has[~is_contra].mean():.3f})")
print(f'Figure 3 saved.')

In [ ]:
# ── Statistical Tests ─────────────────────────────────────────────────────────

from scipy.stats import mannwhitneyu, kruskal, spearmanr
import numpy as np

print('=== Statistical Tests ===\n')

# Kruskal-Wallis: do models differ in contradiction rate?
groups = [df[df['model_prefer'] == m]['contradiction_rate'].dropna().values
          for m in models]
if all(len(g) > 1 for g in groups) and len(groups) > 1:
    all_vals = np.concatenate(groups)
    if np.ptp(all_vals) > 0:
        stat, p = kruskal(*groups)
        print(f'Kruskal-Wallis test (contradiction rate across models):')
        print(f'  H={stat:.3f}, p={p:.4f}')
        print(f'  Interpretation: {"Models differ significantly" if p < 0.05 else "No significant difference"} (α=0.05)')
    else:
        print(f'Kruskal-Wallis test: SKIPPED — all contradiction rates identical ({all_vals[0]:.4f})')

# Spearman correlation: concept validity vs contradiction rate
clean = df.dropna(subset=['concept_valid_rate', 'contradiction_rate'])
if len(clean) > 5 and clean['concept_valid_rate'].std() > 0 and clean['contradiction_rate'].std() > 0:
    rho, p_rho = spearmanr(clean['concept_valid_rate'], clean['contradiction_rate'])
    print(f'\nSpearman ρ (concept validity vs contradiction rate):')
    print(f'  ρ={rho:.3f}, p={p_rho:.4f}')
    print(f'  Interpretation: {"Significant negative correlation" if rho < 0 and p_rho < 0.05 else "No significant correlation"}')
elif len(clean) > 5:
    print(f'\nSpearman ρ: SKIPPED — no variance in one or both variables')

# Trend test: does contradiction rate increase with depth?
all_depth = df_pairs.groupby('depth')['label'].apply(lambda x: (x == 'contradiction').mean())
if len(all_depth) > 2 and all_depth.std() > 0:
    rho_d, p_d = spearmanr(all_depth.index, all_depth.values)
    print(f'\nSpearman ρ (depth vs contradiction rate):')
    print(f'  ρ={rho_d:.3f}, p={p_d:.4f}')
    print(f'  Interpretation: {"Contradiction increases with depth" if rho_d > 0 and p_d < 0.05 else "No significant trend"}')

In [ ]:
# ── Top Contradiction Examples ─────────────────────────────────────────────────

print('=== Top Contradiction Examples ===\n')

top_contra = df_pairs[df_pairs['label'] == 'contradiction'].nlargest(5, 'prob_contradiction')

for idx, row in top_contra.iterrows():
    print(f"Model: {row['model_prefer']} | Depth: {row['depth']} | P(contra): {row['prob_contradiction']:.3f}")
    print(f"Question: {row['question']}")
    print(f"Guards: {row['guards'] or 'none'}")
    print('-' * 70)

# Save enriched pairs for other notebooks
df_pairs.to_json(RESULTS_DIR / 'exp1_pairs_enriched.json', orient='records', indent=2)
df.to_json(RESULTS_DIR / 'exp1_summary_enriched.json', orient='records', indent=2)
print('\nEnriched results saved for Experiments 2, 3, 4.')

## Results Summary

Key findings from this experiment:

1. **Contradiction rate** varies across LLMs — check Table 1 above
2. **Depth effect** — contradiction rate trends upward at later reasoning steps
3. **Concept validity** — higher UMLS concept validity correlates with lower contradiction rate
4. **Guard signals** — `caution_band` and `direction_conflict` fire more often in contradiction pairs

These results go into **Section 4 (Results)** of the paper:
- Table 1 → summary statistics
- Figure 1 → model comparison
- Figure 2 → depth analysis
- Figure 3 → guard signals
- Statistical tests → significance
